In [157]:
# Extensions
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

In [1]:
# Imports
import torch
import pandas as pd
import numpy as np

In [2]:
# Configuration
ID = torch.Tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
x = torch.randn(10, 4, 100)
x_rev_comp = torch.randn(10, 4, 100)
y = torch.randint(0, 2, (10, 1))
batch = (ID, x, x_rev_comp, y)

In [3]:
# Helper functions
def move_module_to_cpu(module):
    module.to("cpu")
    for child in module.children():
        move_module_to_cpu(child)

# Blocks

In [4]:
from eugene.models.base import _blocks as blocks

## Conv1DBlock

In [14]:
conv1d_block = blocks.Conv1DBlock(
    input_len=100,
    input_channels=4,
    output_channels=32,
    conv_kernel=23,
    conv_type="conv1d",
    conv_padding="same",
    pool_type="max",
    norm_type="batchnorm",
    dropout_rate=0.5,
    order="conv-norm-act-dropout-pool"
)
block_out = conv1d_block(x)
conv1d_block, block_out.shape

(Conv1DBlock(
   (layers): Sequential(
     (conv): Conv1d(4, 32, kernel_size=(23,), stride=(1,), padding=same)
     (norm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (act): ReLU()
     (dropout): Dropout(p=0.5, inplace=False)
     (pool): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
   )
 ),
 torch.Size([10, 32, 100]))

In [15]:
conv1d_block = blocks.Conv1DBlock(
    input_len=100,
    input_channels=4,
    output_channels=4,
    conv_kernel=23,
    conv_type="biconv1d",
    conv_padding="same",
    pool_type="max",
    pool_kernel=2,
    norm_type="batchnorm",
    dropout_rate=0.3,
    order="conv-norm-pool"
)
block_out = conv1d_block(x)
conv1d_block, block_out.shape

(Conv1DBlock(
   (layers): Sequential(
     (conv): BiConv1D(4, 4, kernel_size=23, stride=1, padding=same, dilation=1, groups=1, bias=True)
     (norm): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
   )
 ),
 torch.Size([10, 4, 50]))

In [16]:
conv1d_block = blocks.Conv1DBlock(
    input_len=100,
    input_channels=4,
    output_channels=32,
    conv_kernel=None,
    conv_type="inceptionconv1d",
    conv_padding="same",
    pool_type="avg",
    norm_type="layernorm",
    norm_dim=(32, 100),
    dropout_rate=0.5,
    order="conv-norm-act-dropout-pool"
)
block_out = conv1d_block(x)
conv1d_block, block_out.shape

(Conv1DBlock(
   (layers): Sequential(
     (conv): InceptionConv1D(
       (conv1): Conv1d(4, 8, kernel_size=(1,), stride=(1,))
       (conv2_1): Conv1d(4, 8, kernel_size=(1,), stride=(1,))
       (conv2_2): Conv1d(8, 8, kernel_size=(3,), stride=(1,), padding=same)
       (conv3_1): Conv1d(4, 8, kernel_size=(1,), stride=(1,))
       (conv3_2): Conv1d(8, 8, kernel_size=(5,), stride=(1,), padding=same)
       (maxpool): MaxPool1d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
       (conv_maxpool): Conv1d(4, 8, kernel_size=(1,), stride=(1,))
     )
     (norm): LayerNorm((32, 100), eps=1e-05, elementwise_affine=True)
     (act): ReLU()
     (dropout): Dropout(p=0.5, inplace=False)
     (pool): AvgPool1d(kernel_size=(1,), stride=(1,), padding=(0,))
   )
 ),
 torch.Size([10, 32, 100]))

In [17]:
conv1d_block = blocks.Conv1DBlock(
    input_len=100,
    input_channels=4,
    output_channels=32,
    conv_kernel=23,
    conv_type="conv1d",
    conv_padding="same",
    pool_type="max",
    norm_type="batchnorm",
    dropout_rate=0.5,
    order="conv"
)
block_out = conv1d_block(x)
conv1d_block, block_out.shape

(Conv1DBlock(
   (layers): Sequential(
     (conv): Conv1d(4, 32, kernel_size=(23,), stride=(1,), padding=same)
   )
 ),
 torch.Size([10, 32, 100]))

In [19]:
conv1d_block = blocks.Conv1DBlock(
    input_len=100,
    input_channels=4,
    output_channels=32,
    conv_kernel=23,
    conv_type="conv1d",
    conv_padding="valid",
    pool_type="max",
    norm_type="batchnorm",
    dropout_rate=0.5,
    order="pool-conv"
)
block_out = conv1d_block(x)
conv1d_block, block_out.shape

(Conv1DBlock(
   (layers): Sequential(
     (pool): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
     (conv): Conv1d(4, 32, kernel_size=(23,), stride=(1,), padding=valid)
   )
 ),
 torch.Size([10, 32, 78]))

## DenseBlock

In [26]:
dense_block = blocks.DenseBlock(
    input_dim=400, 
    output_dim=1, 
    hidden_dims=[200, 100, 50, 10],
    activations="exponential",
    batchnorm=True,
    batchnorm_first=True, 
    dropout_rates=[0.1, 0.2, None],
    biases=False
)
block_out = dense_block(x.reshape(10, -1))
dense_block, block_out.shape

(DenseBlock(
   (layers): Sequential(
     (0): Linear(in_features=400, out_features=200, bias=False)
     (1): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (2): Exponential()
     (3): Dropout(p=0.1, inplace=False)
     (4): Linear(in_features=200, out_features=100, bias=False)
     (5): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (6): Exponential()
     (7): Dropout(p=0.2, inplace=False)
     (8): Linear(in_features=100, out_features=50, bias=False)
     (9): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (10): Exponential()
     (11): Linear(in_features=50, out_features=10, bias=False)
     (12): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (13): Exponential()
     (14): Linear(in_features=10, out_features=1, bias=False)
   )
 ),
 torch.Size([10, 1]))

## ReccurentBlock

In [28]:
recurrent_block = blocks.RecurrentBlock(
    input_dim=4,
    hidden_dim=10,
    num_layers=2,
    unit_type="lstm",
    bidirectional=True,
    dropout_rates=0.1,
    bias=False,
    batch_first=True
)
block_out = recurrent_block(x.transpose(1, 2))[0]
recurrent_block, block_out.shape

(RecurrentBlock(
   (layers): LSTM(4, 10, num_layers=2, bias=False, batch_first=True, dropout=0.1, bidirectional=True)
 ),
 torch.Size([10, 100, 20]))

In [29]:
recurrent_block = blocks.RecurrentBlock(
    input_dim=4,
    hidden_dim=10,
    num_layers=2,
    unit_type="lstm",
    bidirectional=False,
    dropout_rates=0.1,
    bias=False,
    batch_first=True
)
block_out = recurrent_block(x.transpose(1, 2))[0]
recurrent_block, block_out.shape

(RecurrentBlock(
   (layers): LSTM(4, 10, num_layers=2, bias=False, batch_first=True, dropout=0.1)
 ),
 torch.Size([10, 100, 10]))

In [30]:
recurrent_block = blocks.RecurrentBlock(
    input_dim=4,
    hidden_dim=10,
    num_layers=2,
    unit_type="rnn",
    bidirectional=False,
    dropout_rates=0.1,
    bias=False,
    batch_first=True
)
block_out = recurrent_block(x.transpose(1, 2))[0]
recurrent_block, block_out.shape

(RecurrentBlock(
   (layers): RNN(4, 10, num_layers=2, bias=False, batch_first=True, dropout=0.1)
 ),
 torch.Size([10, 100, 10]))

In [31]:
recurrent_block = blocks.RecurrentBlock(
    input_dim=4,
    hidden_dim=10,
    num_layers=2,
    unit_type="gru",
    bidirectional=False,
    dropout_rates=0.1,
    bias=False,
    batch_first=True
)
block_out = recurrent_block(x.transpose(1, 2))[0]
recurrent_block, block_out.shape

(RecurrentBlock(
   (layers): GRU(4, 10, num_layers=2, bias=False, batch_first=True, dropout=0.1)
 ),
 torch.Size([10, 100, 10]))

# DONE

---

# Scratch